Proyectos bitbucket

In [80]:
%pip install python-dotenv

import requests
import pandas as pd
from dotenv import load_dotenv
import os


# Configura tus credenciales
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

username = os.getenv("BITBUCKET_USERNAME")
app_password = os.getenv("BITBUCKET_APP_PASSWORD")
workspace = os.getenv("BITBUCKET_WORKSPACE")


url_repos = f"https://api.bitbucket.org/2.0/repositories/{workspace}"

response = requests.get(url_repos, auth=(username, app_password))

if response.status_code == 200:
    repositorios = response.json()
    print(f"Repositorios en {workspace}:")
    repos_data = []
    for repo in repositorios.get("values", []):
        repos_data.append(
            {
                "Nombre": repo.get("name"),
                "URL": repo.get("links", {}).get("html", {}).get("href"),
            }
        )
    repos_df = pd.DataFrame(repos_data)
    repos_df = repos_df.sort_values(
        by="Nombre"
    )  # Ordenar por nombre del repositorio en ascendente
    print(repos_df)
else:
    print("Error al obtener repositorios:", response.status_code, response.text)

# Almacenar los repositorios en una variable para uso posterior
repositorios = [
    repo["Nombre"]
    for repo in repos_data
    if repo["Nombre"] not in ["pgp", "Pruebas_erp", "Inventario"]
]
print(repositorios)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Repositorios en itmz:
              Nombre                                           URL
4       ApiEmpleados       https://bitbucket.org/itmz/apiempleados
8         Inventario         https://bitbucket.org/itmz/inventario
9        Pruebas_erp        https://bitbucket.org/itmz/pruebas_erp
7       api_data_ERP       https://bitbucket.org/itmz/api_data_erp
1  api_it_management  https://bitbucket.org/itmz/api_it_management
3                b2b                https://bitbucket.org/itmz/b2b
2                b2c                https://bitbucket.org/itmz/b2c
6                efi                https://bitbucket.org/itmz/efi
5                fip                https://bitbucket.org/itmz/fip
0                pgp                https://bitbucket.org/itmz/pgp
['api_it_management', 'b2c', 'b2b', 'ApiEmpleados', 'fip', 'efi', 'api_data_ERP']


In [90]:
from IPython.display import display
import json


# Definir una función para obtener pull requests de un repositorio en cualquier estado
def obtener_pull_requests(repo_slug, page=1, pagelen=20):
    url_pulls = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests?state=ALL"
    params = {"pagelen": pagelen, "page": page}
    response_pulls = requests.get(
        url_pulls, params=params, auth=(username, app_password)
    )

    # print(
    #     json.dumps(
    #         json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")
    #     )
    # )

    print(f"Repo: {repo_slug}")
    # print(f"Cantidad registros: {len(response_pulls.json().get('values', []))}")
    # print(f"Total: {response_pulls.json().get('size', 0)}\n")

    if response_pulls.status_code == 200:
        pulls = response_pulls.json()
        pulls_data = []
        for pull in pulls.get("values", []):
            pulls_data.append(
                {
                    "ID": pull.get("id"),
                    "Repositorio": repo_slug,
                    "Branch": pull.get("source", {}).get("branch", {}).get("name"),
                    "Autor": pull.get("author", {}).get("display_name"),
                    "Fecha de Creación": pd.to_datetime(pull.get("created_on"))
                    .tz_convert("America/Bogota")
                    .strftime("%d-%m-%Y"),
                    "Días abierto": (
                        pd.Timestamp.now(tz="America/Bogota")
                        - pd.to_datetime(pull.get("created_on")).tz_convert(
                            "America/Bogota"
                        )
                    ).days,
                    "Estado": pull.get("state"),
                    "Cant. comentarios": pull.get("comment_count", 0),
                    "created_on": pd.to_datetime(pull.get("created_on")),
                }
            )
        return pd.DataFrame(pulls_data)
    else:
        print(
            f"Error al obtener pull requests para {repo_slug}:",
            response_pulls.status_code,
            response_pulls.text,
        )
        return pd.DataFrame()


# Crear un DataFrame con los pull requests de todos los repositorios

# for repo_slug in ["fip"]:
#     pulls = obtener_pull_requests(repo_slug)
#     todos_pulls = pd.concat([todos_pulls, pulls], ignore_index=False)

df = pd.DataFrame()
# for repo_slug in repositorios:
for repo_slug in ["fip", "b2b"]:
    pulls = obtener_pull_requests(repo_slug)
    df = pd.concat([df, pulls], ignore_index=True)

df_ordered_asc = df.sort_values(by="created_on", ascending=False)
df = df_ordered_asc

# fip_pulls = obtener_pull_requests("fip")
print(f"Cantidad de registros df: {len(df)}")


# Ordenar los registros por la fecha de creación de forma descendente
# todos_pulls = todos_pulls.sort_values(by="Fecha de Creación", ascending=False)


print("Pull Requests en todos los repositorios:")
display(df.head(50))


Repo: fip
Repo: b2b
Cantidad de registros df: 40
Pull Requests en todos los repositorios:


,ID,Repositorio,Branch,Autor,Fecha de Creación,Días abierto,Estado,Cant. comentarios,created_on
20,620,b2b,feature/BTB-730,Camila Castañeda,27-02-2025,0,OPEN,0,2025-02-27 22:22:47.001839+00:00
26,619,b2b,feature/BTB-640,Camila Castañeda,27-02-2025,1,OPEN,0,2025-02-27 19:47:48.704626+00:00
22,618,b2b,feature/BTB-689,Camila Castañeda,27-02-2025,1,OPEN,0,2025-02-27 14:30:54.100646+00:00
1,765,fip,feature/CPK-897,Heriberto Sanchez,26-02-2025,1,OPEN,9,2025-02-26 21:55:53.332634+00:00
23,617,b2b,feature/BTB-726,Camila Castañeda,26-02-2025,2,MERGED,0,2025-02-26 15:25:24.944264+00:00
24,616,b2b,feature/BTB-728,Camila Castañeda,26-02-2025,2,MERGED,0,2025-02-26 14:54:26.516375+00:00
3,764,fip,feature/CPK-906,Camila Castañeda,26-02-2025,2,OPEN,0,2025-02-26 14:12:14.785006+00:00
21,615,b2b,feature/BTB-735,Andrés Felipe Perdomo,24-02-2025,4,MERGED,0,2025-02-24 19:28:08.674938+00:00
2,763,fip,feature/CPK-920,Heriberto Sanchez,24-02-2025,4,MERGED,4,2025-02-24 19:16:10.831349+00:00
4,762,fip,feature/CPK-892,Camilo Medina,21-02-2025,7,MERGED,8,2025-02-21 18:25:03.499392+00:00
